In [1]:
# Import Libraries
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
# Read Campaigns CSV as df
df = pd.read_csv("campaigns.csv")

In [3]:
### Sort Weekday by the Open Rate where the total recipients are greater than 10


# Create a column thaat converts the Open Rate percentages to decimals
df['Open Rate Value'] = df['Open Rate'].str.rstrip('%').astype('float') / 100

# Create a column thaat converts the Click Rate percentages to decimals
df['Click Rate Value'] = df['Click Rate'].str.rstrip('%').astype('float') / 100

# Create a DF for only emails that have at least 10 recipients
df_tr_10 = df[df['Total Recipients'] > 10]

# Series that Sorts the mean Open Rate for each Weekday
weekday_by_or = df_tr_10.groupby('Send Weekday')['Open Rate Value'].mean().sort_values(ascending=False)
weekday_by_or

Send Weekday
Thursday     0.638300
Friday       0.609100
Monday       0.606747
Tuesday      0.568940
Wednesday    0.559717
Name: Open Rate Value, dtype: float64

In [4]:
'''
Thursday, Friday, Monday, Tuesday, Wednesday -> these are the weekdays' open rate in descending order

As we can see, generally having email on the bookend of the week tend to be more effective, 
as this is the time where most people check their emails because they are either just starting their week and need to check their emails, 
or they are just finishing their week and have time to check their email before their week starts back up again

This is also slightly misleading because the two highest days: Thursday and Friday have only one value, where every other weekday has multiple values.

'''

"\nThursday, Friday, Monday, Tuesday, Wednesday -> these are the weekdays' open rate in descending order\n\nAs we can see, generally having email on the bookend of the week tend to be more effective, \nas this is the time where most people check their emails because they are either just starting their week and need to check their emails, \nor they are just finishing their week and have time to check their email before their week starts back up again\n\nThis is also slightly misleading because the two highest days: Thursday and Friday have only one value, where every other weekday has multiple values.\n\n"

In [5]:
### Sort Weekday by the Click Rate where the total recipients are greater than 10

# Series that Sorts the mean Click Rate for each Weekday
weekday_by_cr = df_tr_10.groupby('Send Weekday')['Click Rate Value'].mean().sort_values(ascending=False)
weekday_by_cr

Send Weekday
Thursday     0.117000
Friday       0.090900
Monday       0.074553
Tuesday      0.026360
Wednesday    0.006033
Name: Click Rate Value, dtype: float64

In [6]:
'''
extremely similar to the Open Rate Data
'''

'\nextremely similar to the Open Rate Data\n'

In [7]:
### Does Time of Day matter at all?

# Convert to Datetime variable
df['DateTime'] = pd.to_datetime(df['Send Date'], format='%b %d, %Y %I:%M %p')

## Open/Click Rates based on time of day

In [8]:
# 1am-Noon Open
morning_or = df[(df['DateTime'].dt.hour >= 1) & (df['DateTime'].dt.hour < 12)]['Open Rate Value'].mean()
morning_or

0.63714

In [9]:
# 1am-Noon Click
morning_cr = df[(df['DateTime'].dt.hour >= 1) & (df['DateTime'].dt.hour < 12)]['Click Rate Value'].mean()
morning_cr

0.20724

In [10]:
# Noon-5pm Open
afternoon_or = df[(df['DateTime'].dt.hour >= 12) & (df['DateTime'].dt.hour < 17)]['Open Rate Value'].mean()
afternoon_or

0.5850142857142858

In [11]:
# Noon-5pm Click
afternoon_cr = df[(df['DateTime'].dt.hour >= 12) & (df['DateTime'].dt.hour < 17)]['Click Rate Value'].mean()
afternoon_cr

0.04221428571428571

In [12]:
# 5-9pm Open
evening_or = df[(df['DateTime'].dt.hour >= 17) & (df['DateTime'].dt.hour < 21)]['Open Rate Value'].mean()
evening_or

0.6171666666666666

In [13]:
# 5-9pm Click
evening_cr = df[(df['DateTime'].dt.hour >= 17) & (df['DateTime'].dt.hour < 21)]['Click Rate Value'].mean()
evening_cr

0.06086666666666666

In [14]:
# 9-1am Open
night_or = df[(df['DateTime'].dt.hour >= 21) | (df['DateTime'].dt.hour <= 1)]['Open Rate Value'].mean()
night_or

0.603491304347826

In [15]:
# 9-1am Click
night_cr = df[(df['DateTime'].dt.hour >= 21) | (df['DateTime'].dt.hour <= 1)]['Click Rate Value'].mean()
night_cr

0.08078260869565217

In [16]:
'''
Overall, the time of day seems to have very little impact on the open rate of the emails for each week. Much of this is proably due to there being no data column on how quickly these opens occur, essentially accounting all opens the same regardless of promptness.
To test which times are the most effective it would be best to have an extra column detailing how quickly these recipients clicked on the respective email.
In general, it seems that most of the people that check their Gracepoint email will everntually get to their email at some point in the day or the next day.
However, the least effective time seems to be in the afternoon, as this is the only time of day that does not eclipse an open rate of 60% - at 58.5%.

As the hours in the day increase, the attentiveness in regards to click rate in the email has a staggering difference between morning emails and email at any other times. 

Starting at around 20.724% click rate for morning emails, then this number was followed by a 4.22% click rate for afternoon emails, 6.09% click rate for evening emails, and 8.08% click rate for night emails.
'''

'\nOverall, the time of day seems to have very little impact on the open rate of the emails for each week. Much of this is proably due to there being no data column on how quickly these opens occur, essentially accounting all opens the same regardless of promptness.\nTo test which times are the most effective it would be best to have an extra column detailing how quickly these recipients clicked on the respective email.\nIn general, it seems that most of the people that check their Gracepoint email will everntually get to their email at some point in the day or the next day.\nHowever, the least effective time seems to be in the afternoon, as this is the only time of day that does not eclipse an open rate of 60% - at 58.5%.\n\nAs the hours in the day increase, the attentiveness in regards to click rate in the email has a staggering difference between morning emails and email at any other times. \n\nStarting at around 20.724% click rate for morning emails, then this number was followed b

In [17]:
### What students (emails/computing ids) check their email the most

data_dir = Path.cwd() / "granular_activity opens"

dfs = []

for i in data_dir.iterdir():
    df_opens = pd.read_csv(i)  
    #df["Date"] = pd.to_datetime(df["Date"])
    #df.set_index("dt", inplace=True)
    
    ticker = i.stem 
    df_opens["ticker"] = ticker
    
    dfs.append(df_opens)

df_opens = pd.concat(dfs)

df_opens.columns = df_opens.columns.str.lower()

# Subset data to only include UVA student emails
uva_emails = df_opens[df_opens['email'].str.endswith('@virginia.edu')]

# Add "Computing ID" column to exclude the full email
uva_emails['computing id'] = uva_emails['email'].str.replace('@virginia.edu', '')

# Number of Opens per UVA Computing ID over the course of the year
uva_cid_opens = uva_emails['computing id'].value_counts()

### Which recipients almost always open emails (> 20 opens)
greater_than_20_opens = uva_cid_opens[uva_cid_opens > 20]
greater_than_20_opens

/var/folders/lt/bpzfs8lx2l9frgz0m351c2kw0000gn/T/ipykernel_56941/264243483.py:25: FutureWarning: The default value of regex will change from True to False in a future version.
  uva_emails['computing id'] = uva_emails['email'].str.replace('@virginia.edu', '')
/var/folders/lt/bpzfs8lx2l9frgz0m351c2kw0000gn/T/ipykernel_56941/264243483.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uva_emails['computing id'] = uva_emails['email'].str.replace('@virginia.edu', '')


rgp3qv     293
qce3fh     126
adw5ke     117
mfG9RV     112
awp8xj      94
spb2jt      86
fhc5mf      82
dcf5kwt     80
ybz3vj      78
wst5qx      70
wsa2dz      68
dkh2ns      65
fkm5qkx     64
mxz6kx      58
ajl5uu      57
llp5ww      55
nkd6gq      51
pdq8rn      51
efd3tr      49
yaj2jf      48
awk8eup     48
rjv9ku      47
mqx4vk      47
ecr8yt      42
xzc7xd      40
aan3gds     35
cfg6cp      35
rxb9tm      34
zga9sh      34
eqb3tw      32
vfr4fd      31
eta8bm      30
umt6cs      28
jea5pb      28
ktc8uq      27
xhx5mq      25
dva4ny      25
ik4vrb      25
syq8ft      24
hmo3yvm     24
msw3jg      24
zux4wh      23
zmc4vg      22
Name: computing id, dtype: int64

In [18]:
'''
These are the students that check their emails the most frequently and would typically be the most trustworthy in delievering information/updates to.
'''

'\nThese are the students that check their emails the most frequently and would typically be the most trustworthy in delievering information/updates to.\n'

In [19]:
### Which recipients almost never open emails (> 10 opens)
less_than_10_opens = uva_cid_opens[uva_cid_opens < 10]
less_than_10_opens

sdr2dm     9
nyd6bx     9
dvy4kv     9
ntn3wm     8
msr9vb     8
byz7ex     8
jcw4xjq    7
pfp3zp     7
gzg8pf     6
jku8pk     6
dqb3ft     6
dhc5qr     6
pmh3ax     6
duk8su     5
rag6sk     5
yay6mu     5
jxc9zq     4
gww7fp     4
ngf5nf     4
dug2vy     4
dhv3ea     3
mxj5rk     3
wrb8ky     2
rmy7wn     2
gcb9eu     1
Name: computing id, dtype: int64

In [20]:
'''
These are the students' computing ids that frequently neglect their email.
Of all the recipients, these would be the students that we may consider removing from teh email list as they are not getting any added benefits of having the email sent to them weekly.
'''

"\nThese are the students' computing ids that frequently neglect their email.\nOf all the recipients, these would be the students that we may consider removing from teh email list as they are not getting any added benefits of having the email sent to them weekly.\n"

In [21]:
### Over the Year, which events got the most clicks (ranked)

data_dir = Path.cwd() / "granular_activity clicks"

dfs = []

for i in data_dir.iterdir():
    df_clicks = pd.read_csv(i)  
    #df["Date"] = pd.to_datetime(df["Date"])
    #df.set_index("dt", inplace=True)
    
    ticker = i.stem 
    df_clicks["ticker"] = ticker
    
    dfs.append(df_clicks)

df_clicks = pd.concat(dfs)

df_clicks.columns = df_clicks.columns.str.lower()

event_clicks = df_clicks["ticker"].value_counts()

greater_than_10_clicks = event_clicks[event_clicks >= 10]
greater_than_10_clicks

15074446_gp-c-ville-11-14-weekly-update        48
15510646_gp-c-ville-1-17-weekly-update         25
15508366_gp-c-ville-update-                    24
15021713_week-of-9-12-update-                  24
15512278_gp-c-ville-2-13-weekly-update         24
15023657_week-10-3-gp-c-ville-updates          16
15024977_gp-c-ville-week-10-17-updates         16
15020165_carter-mountain-trip-8-27-campaign    15
15074914_gp-c-ville-11-21-weekly-update        15
15515878_gp-c-ville-4-10-weekly-update         10
15073962_gp-c-ville-11-7-weekly-update         10
Name: ticker, dtype: int64

In [22]:
'''
In regards to email attentiveness, these were the events (in order) that seemed to have the biggest success over the course of the 2022-2023 school year.
May be attributed to the specific event?
Or possibly the graphic? (Color, fonts, design?)
Or possibly the advertisement of the event during service?
'''

'\nIn regards to email attentiveness, these were the events (in order) that seemed to have the biggest success over the course of the 2022-2023 school year.\nMay be attributed to the specific event?\nOr possibly the graphic? (Color, fonts, design?)\nOr possibly the advertisement of the event during service?\n'